Data processing

In [1]:
!pip install --user transformers
!pip install --user tensorflow==2.4.1
!pip install --user sklearn
!pip install --user keras

In [2]:
import pandas as pd
import ast
from transformers import BertTokenizerFast,TFBertModel
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Model
from keras.callbacks import EarlyStopping,ModelCheckpoint

ImportError: Traceback (most recent call last):
  File "C:\Users\nango\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\pywrap_tensorflow.py", line 64, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed: A dynamic link library (DLL) initialization routine failed.


Failed to load the native TensorFlow runtime.

See https://www.tensorflow.org/install/errors

for some common reasons and solutions.  Include the entire stack trace
above this error message when asking for help.

In [4]:
import zipfile
with zipfile.ZipFile("spanbert_hf_base.zip","r") as zip_ref:
    zip_ref.extractall()

In [3]:
max_len=400
doc_max_len=400

In [4]:
train_data=pd.read_csv("https://raw.githubusercontent.com/ipavlopoulos/toxic_spans/master/data/tsd_train.csv")
test_data=pd.read_csv("https://raw.githubusercontent.com/ipavlopoulos/toxic_spans/master/data/tsd_test.csv")
train, validation = train_test_split(train_data, test_size = 0.1,random_state=123)

In [5]:
def process_df(df):
    df.spans=df.spans.apply(lambda x: ast.literal_eval(x))
    df["sentiment"]=df.spans.apply(lambda x: "Negative" if len(x)>0 else "Neutral")
    return df

In [6]:
train=process_df(train)
validation=process_df(validation)
test_data=process_df(test_data)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
def find_spans(span_index,text):
    '''Function takes in the span index list and returns the text those indices correspond to'''
    try:  
        span_index=[int(item) for item in span_index]
        current_index=span_index[0]
        spans=[]
        span_text=[]
        for i in range(len(span_index)-1):
            if span_index[i+1]-span_index[i]>1:
                spans.append((current_index,span_index[i]+1))
                current_index=span_index[i+1]
        if current_index==span_index[0]:
            spans.append((current_index,span_index[-1]+1))
        return spans
    except:
        return []

Implementing SpanBERT

In [11]:
spanbert_path='spanbert_hf_base/'

In [12]:
tokenizer = BertTokenizerFast.from_pretrained(spanbert_path)

In [13]:
class TrainTweetExamples:
    def __init__(self,text,sentiment,span_index,max_len,doc_max_len):
        self.text=text
        self.sentiment=sentiment
        self.span_index=span_index
        self.character_in_ans=None
        self.max_len=max_len
        self.doc_max_len=doc_max_len
    
    def preprocess(self):
        text=self.text
        sentiment=self.sentiment
        span_index=self.span_index

        # Mark character indices in text:
        character_in_ans=[0]*len(text)
        spans=find_spans(span_index,text)
        for start,end in spans: # marking the 1s
              for idx in range(start,end):
                    character_in_ans[idx]=1


        # Text tokenization and input prep
        tokenized_text = tokenizer.encode_plus(text, return_offsets_mapping=True, max_length = self.max_len)
        tokenized_sentiment=tokenizer.encode_plus(sentiment, return_offsets_mapping=True, max_length = self.max_len)
        input_ids = tokenized_text.input_ids + tokenized_sentiment.input_ids[1:] # The choice of 1: is to remove the beginning 101.
        token_type_ids = [0] * len(tokenized_text.input_ids) + [1] * len(
                tokenized_sentiment.input_ids[1:]
            )
        attention_mask = [1] * len(input_ids)

        # Generate the actual tokens:
        ans_token_idx = []
        for idx, (start, end) in enumerate(tokenized_text.offset_mapping):
            if sum(character_in_ans[start:end]) > 0:
                ans_token_idx.append([0,1])
            else: 
                ans_token_idx.append([1,0]) # Not toxic

        # Padding
        padding_length = self.max_len - len(input_ids)
        if padding_length > 0: 
            input_ids = input_ids + ([0] * padding_length)
            attention_mask = attention_mask + ([0] * padding_length)
            token_type_ids = token_type_ids + ([0] * padding_length)
        padding_length_toxic=self.doc_max_len - len(ans_token_idx)
        if padding_length_toxic>0:
            ans_token_idx=ans_token_idx+[[0,0]]*padding_length_toxic


        #Assignments
        self.input_ids = input_ids
        self.token_type_ids = token_type_ids
        self.attention_mask = attention_mask
        self.context_token_to_char = tokenized_text.offset_mapping
        self.character_in_ans=character_in_ans
        self.ans_token_idx = ans_token_idx

    def check_length(self):
        print("Length of tokenized input: ",len(self.tokenized_text.input_ids),", length of tokenized answers: ",len(self.ans_token_idx))

In [14]:
def create_tweet_examples(data,max_len,doc_max_len):
    tweet_examples = []
    count = 0
    for index, row in data.iterrows():
        sentiment = row['sentiment']
        text = row['text']
        span_index=row['spans']
        ## turn the data into TrainTweetExample objects
        try:
            tweet_eg = TrainTweetExamples(text, sentiment, span_index,max_len,doc_max_len)
            tweet_eg.preprocess()
            tweet_examples.append(tweet_eg)
        except: ## keep track of the number that can't be tokenized/processed
            count += 1 
    print("Couldn't process",count,"points")
    return tweet_examples

In [15]:
def create_inputs_targets(tweet_examples):
    dataset_dict = {
        "input_ids": [],
        "token_type_ids": [],
        "attention_mask": [],
        "ans_token_idx":[]
    }
    for item in tweet_examples:
        #if item.skip == False:
        for key in dataset_dict.keys():
              dataset_dict[key].append(getattr(item, key))

    x = [
        np.array(dataset_dict["input_ids"]),
        np.array(dataset_dict["token_type_ids"]),
        np.array(dataset_dict["attention_mask"])]
    y = np.array(dataset_dict["ans_token_idx"])
    return x, y

In [16]:
train_tweet_examples = create_tweet_examples(train,max_len,doc_max_len)
x_train, y_train = create_inputs_targets(train_tweet_examples)
val_tweet_examples = create_tweet_examples(validation,max_len,doc_max_len)
x_val, y_val = create_inputs_targets(val_tweet_examples)
test_tweet_examples = create_tweet_examples(test_data,max_len,doc_max_len)
x_test, y_test = create_inputs_targets(test_tweet_examples)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Couldn't process 0 points
Couldn't process 0 points
Couldn't process 0 points


Creating the model

In [17]:
def create_model(max_len,doc_max_len):
    encoder = TFBertModel.from_pretrained(spanbert_path,from_pt=True)
    input_ids = layers.Input(shape=(max_len ,), dtype=tf.int32)
    token_type_ids = layers.Input(shape=(max_len,), dtype=tf.int32)
    attention_mask = layers.Input(shape=(max_len,), dtype=tf.int32)
    start_logits = encoder(
             input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask
        )[0]
    #start_logits=layers.Bidirectional(layers.GRU(256,return_sequence=True))(start_logits)
    #start_logits=layers.Conv1D(128,2,padding="same")(start_logits)
    start_logits=layers.Dropout(0.2)(start_logits)
    start_logits=layers.TimeDistributed(layers.Dense(2,activation="softmax"))(start_logits)
    model=Model([input_ids,token_type_ids,attention_mask],start_logits)
    return model

In [18]:
use_tpu = False
if use_tpu:
    # Create distribution strategy
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)

    # Create model
    with strategy.scope():
        model = create_model(max_len,doc_max_len)
else:
    model = create_model(max_len,doc_max_len)

All PyTorch model weights were used when initializing TFBertModel.

Some weights or buffers of the TF 2.0 model TFBertModel were not initialized from the PyTorch model and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

In [19]:
optimizer = keras.optimizers.Adam(lr=5e-5)
model.compile(optimizer=optimizer,loss="categorical_crossentropy",metrics=['accuracy'])

In [20]:
early_stop=EarlyStopping(monitor="val_accuracy",patience=3,restore_best_weights=True)
model_checkpoint=ModelCheckpoint(filepath="NN_SpanBERT_TC",monitor="val_accuracy",save_best_only=True)

In [22]:
model.fit(x_train,y_train,epochs=3,batch_size=8,validation_split=0.10,callbacks=[early_stop,model_checkpoint])

Epoch 1/3
804/804 [==============================] - 1124s 1s/step - loss: 0.0259 - accuracy: 0.9913 - val_loss: 0.0214 - val_accuracy: 0.9937


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Span NN materials/NN_SpanBERT_TC/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Span NN materials/NN_SpanBERT_TC/assets


Epoch 2/3
804/804 [==============================] - 1119s 1s/step - loss: 0.0227 - accuracy: 0.9925 - val_loss: 0.0206 - val_accuracy: 0.9937
Epoch 3/3
  7/804 [..............................] - ETA: 17:47 - loss: 0.0299 - accuracy: 0.9878

KeyboardInterrupt: ignored

Predicting

In [23]:
def get_indices(x):
    indice_prediction=[]
    for single_prediction in x:
        indice_prediction.append([idx for idx,value in enumerate(single_prediction) if value[1]>=0.5])
        return indice_prediction

In [24]:
def get_indices_final(indice_prediction,tweet_examples):
    all_correct_indices=[]
    for i in range(len(indice_prediction)):
    correct_indices=[]
    indices=indice_prediction[i]
    tweet=tweet_examples[i].context_token_to_char
    for i in indices:
        if i <len(tweet):
            start,end=tweet[i]
            correct_indices.extend([index for index in range(start,end)])
    all_correct_indices.append(correct_indices)
    return all_correct_indices

In [25]:
def convert_to_indices(input_data,tweet_examples):
    global model
    predictions=model.predict(input_data)
    indice_prediction=get_indices(predictions)
    final_indices=get_indices_final(indice_prediction,tweet_examples)
    return final_indices

In [26]:
def f1(predictions, gold):
    """
    F1 (a.k.a. DICE) operating on two lists of offsets (e.g., character).
    >>> assert f1([0, 1, 4, 5], [0, 1, 6]) == 0.5714285714285714
    :param predictions: a list of predicted offsets
    :param gold: a list of offsets serving as the ground truth
    :return: a score between 0 and 1
    """
    if len(gold) == 0:
        return 1. if len(predictions) == 0 else 0.
    if len(predictions) == 0:
        return 0.
    predictions_set = set(predictions)
    gold_set = set(gold)
    nom = 2 * len(predictions_set.intersection(gold_set))
    denom = len(predictions_set) + len(gold_set)
    return float(nom)/float(denom)

In [27]:
def all_f1_scores(predictions,golds):
    f1_scores=[]
    for i in range(len(predictions)):
        f1_score=f1(predictions[i],golds.spans.values[i])
        f1_scores.append(f1_score)
    return f1_scores

In [28]:
indices_val=convert_to_indices(x_val,val_tweet_examples)
f1_val=all_f1_scores(indices_val,validation)
indices_test=convert_to_indices(x_test,test_tweet_examples)
f1_test=all_f1_scores(indices_test,test_data)

In [29]:
df_val=pd.DataFrame({"span":indices_val,"text":validation.text.values})
df_test=pd.DataFrame({"span":indices_test,"text":test_data.text.values})

In [30]:
df_val.to_csv("Spanbert_TC_validation.csv")
df_test.to_csv("Spanbert_TC_test.csv")

0.842483931123918